In [1]:
import pyspark 
sc = pyspark.SparkContext('local[*]')
# do something to prove it works
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 5)


[121, 302, 980, 102, 953]

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Read CSV in Jupyter with PySpark") \
    .getOrCreate()

In [7]:
accountsDF = spark.read.csv("files/accounts.csv", header=True, inferSchema=True)
print(accountsDF.count())

500000


In [8]:
countryAbbreviationDF = spark.read.csv("files/country_abbreviation.csv", header=True, inferSchema=True)
print(accountsDF.count())

500000


In [9]:
transactionsDF = spark.read.csv("files/transactions.csv", header=True, inferSchema=True)
print(accountsDF.count())

500000
